In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tqdm
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail
import requests
from nba_api.stats.static.players import find_players_by_full_name
from adjustText import adjust_text
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import time
from PIL import Image
from collections import defaultdict

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2

from nba_api.stats.endpoints.playbyplay import PlayByPlay
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name
from nba_api.stats.endpoints.playergamelogs import PlayerGameLogs
from nba_api.stats.endpoints.boxscoretraditionalv2 import BoxScoreTraditionalV2
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail

from nba_api.stats.endpoints.leaguedashteamshotlocations import LeagueDashTeamShotLocations
from nba_api.stats.endpoints.leaguedashplayerptshot import LeagueDashPlayerPtShot

from nba_api.stats.endpoints.playerdashboardbyshootingsplits import PlayerDashboardByShootingSplits
from nba_api.stats.endpoints.synergyplaytypes import SynergyPlayTypes
from nba_api.stats.endpoints.leaguedashptstats import LeagueDashPtStats
from nba_api.stats.endpoints.leaguedashplayerstats import LeagueDashPlayerStats

from nba_api.stats.endpoints.leaguehustlestatsplayer import LeagueHustleStatsPlayer
from nba_api.stats.endpoints.leaguedashteamstats import LeagueDashTeamStats
from nba_api.stats.endpoints.leaguehustlestatsteam import LeagueHustleStatsTeam
from nba_api.stats.endpoints.leaguedashteamstats import LeagueDashTeamStats
from nba_api.stats.endpoints.synergyplaytypes import SynergyPlayTypes
from nba_api.stats.endpoints.leaguedashteamptshot import LeagueDashTeamPtShot

In [4]:
very_tight_shots = "0-2 Feet - Very Tight"
tight_shots = "2-4 Feet - Tight"
open_shots = "4-6 Feet - Open"
wide_open_shots = "6+ Feet - Wide Open"

In [ ]:

https://stats.nba.com/stats/leaguedashteamptshot?CloseDefDistRange=2-4+Feet+-+Tight&College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&DribbleRange=&GameScope=&GameSegment=&GeneralRange=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=1&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2021-22&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&ShotDistRange=&StarterBench=&TeamID=0&TouchTimeRange=&VsConference=&VsDivision=&Weight=

In [5]:
distances = {
    'VERY TIGHT': very_tight_shots,
    'TIGHT': tight_shots,
    'OPEN': open_shots,
    'WIDE OPEN': wide_open_shots
}

In [6]:
from datetime import date, timedelta


In [7]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [9]:
start_date = date(2021, 10, 19)
end_date = date(2022, 2, 4)
for single_date in daterange(start_date, end_date):
    print(single_date.strftime("%d/%m/%Y"))

19/10/2021
20/10/2021
21/10/2021
22/10/2021
23/10/2021
24/10/2021
25/10/2021
26/10/2021
27/10/2021
28/10/2021
29/10/2021
30/10/2021
31/10/2021
01/11/2021
02/11/2021
03/11/2021
04/11/2021
05/11/2021
06/11/2021
07/11/2021
08/11/2021
09/11/2021
10/11/2021
11/11/2021
12/11/2021
13/11/2021
14/11/2021
15/11/2021
16/11/2021
17/11/2021
18/11/2021
19/11/2021
20/11/2021
21/11/2021
22/11/2021
23/11/2021
24/11/2021
25/11/2021
26/11/2021
27/11/2021
28/11/2021
29/11/2021
30/11/2021
01/12/2021
02/12/2021
03/12/2021
04/12/2021
05/12/2021
06/12/2021
07/12/2021
08/12/2021
09/12/2021
10/12/2021
11/12/2021
12/12/2021
13/12/2021
14/12/2021
15/12/2021
16/12/2021
17/12/2021
18/12/2021
19/12/2021
20/12/2021
21/12/2021
22/12/2021
23/12/2021
24/12/2021
25/12/2021
26/12/2021
27/12/2021
28/12/2021
29/12/2021
30/12/2021
31/12/2021
01/01/2022
02/01/2022
03/01/2022
04/01/2022
05/01/2022
06/01/2022
07/01/2022
08/01/2022
09/01/2022
10/01/2022
11/01/2022
12/01/2022
13/01/2022
14/01/2022
15/01/2022
16/01/2022
17/01/2022

In [36]:
old_data = pd.read_csv('shot_types_found/shot_types_new.csv')

In [37]:
data = []

start_date = date(2022, 1, 8)
end_date = date(2022, 2, 4)
#end_date = date(2021, 10, 20)
for single_date in daterange(start_date, end_date):
    target_date = single_date.strftime("%m/%d/%Y")
    for period in [1, 2, 3, 4]:
        #target_date = '02/02/2022'
        print(target_date)
        quarter_stats = LeagueDashTeamStats(period=period, date_from_nullable=target_date, date_to_nullable=target_date).get_data_frames()[0]
        time.sleep(0.6)
        for key in distances:
            distance = distances[key]
            shots = LeagueDashTeamPtShot(close_def_dist_range_nullable=distance, period_nullable=period, date_from_nullable=target_date, date_to_nullable=target_date).get_data_frames()[0]
            merged_df = pd.merge(left=quarter_stats, right=shots, on=('TEAM_ID', 'TEAM_NAME'), suffixes=('_NORMAL', '_SHOT'))
            equal = merged_df.loc[merged_df.FG3A_NORMAL == merged_df.FG3A_SHOT]
            if len(equal) > 0:
                #print(period, key)
                #print(equal)
                for i in range(len(equal)):
                    cur_row = equal.iloc[i]
                    splitted = target_date.split("/")
                    #2022-02-02
                    data.append(
                        {
                            'GAME_DATE': f"{splitted[2]}-{splitted[0]}-{splitted[1]}",
                            'TEAM_ID': cur_row.TEAM_ID,
                            'TEAM_NAME': cur_row.TEAM_NAME,
                            'PERIOD': period,
                            'SHOT_TYPE': distances[key],
                            'FG3M': cur_row.FG3M_NORMAL,
                            'FG3A': cur_row.FG3A_NORMAL,
                        }
                    )
            #if len(shots) > 0:
            #    print(period, key)
            #    print(shots)
            time.sleep(0.6)

df = pd.DataFrame(data)

01/08/2022
01/08/2022
01/08/2022
01/08/2022
01/09/2022
01/09/2022
01/09/2022
01/09/2022
01/10/2022
01/10/2022
01/10/2022
01/10/2022
01/11/2022
01/11/2022
01/11/2022
01/11/2022
01/12/2022
01/12/2022
01/12/2022
01/12/2022
01/13/2022
01/13/2022
01/13/2022
01/13/2022
01/14/2022
01/14/2022
01/14/2022
01/14/2022
01/15/2022
01/15/2022
01/15/2022
01/15/2022
01/16/2022
01/16/2022
01/16/2022
01/16/2022
01/17/2022
01/17/2022
01/17/2022
01/17/2022
01/18/2022
01/18/2022
01/18/2022
01/18/2022
01/19/2022
01/19/2022
01/19/2022
01/19/2022
01/20/2022
01/20/2022
01/20/2022
01/20/2022
01/21/2022
01/21/2022
01/21/2022
01/21/2022
01/22/2022
01/22/2022
01/22/2022
01/22/2022
01/23/2022
01/23/2022
01/23/2022
01/23/2022
01/24/2022
01/24/2022
01/24/2022
01/24/2022
01/25/2022
01/25/2022
01/25/2022
01/25/2022
01/26/2022
01/26/2022
01/26/2022
01/26/2022
01/27/2022
01/27/2022
01/27/2022
01/27/2022
01/28/2022
01/28/2022
01/28/2022
01/28/2022
01/29/2022
01/29/2022
01/29/2022
01/29/2022
01/30/2022
01/30/2022
01/30/2022

In [38]:
df = pd.DataFrame(data)

In [39]:
full_df = pd.concat([old_data, df], ignore_index=True)

In [45]:
full_df.sort_values(by='FG3A', ascending=False)

,GAME_DATE,TEAM_ID,TEAM_NAME,PERIOD,SHOT_TYPE,FG3M,FG3A
49,2021-12-26,1610612739,Cleveland Cavaliers,2,6+ Feet - Wide Open,3,11
18,2021-11-10,1610612766,Charlotte Hornets,3,6+ Feet - Wide Open,3,10
68,2022-01-19,1610612760,Oklahoma City Thunder,1,6+ Feet - Wide Open,3,10
35,2021-12-02,1610612749,Milwaukee Bucks,1,6+ Feet - Wide Open,2,10
59,2022-01-03,1610612742,Dallas Mavericks,2,6+ Feet - Wide Open,2,9
28,2021-11-22,1610612763,Memphis Grizzlies,4,6+ Feet - Wide Open,3,9
53,2021-12-29,1610612754,Indiana Pacers,2,6+ Feet - Wide Open,4,9
37,2021-12-03,1610612753,Orlando Magic,3,6+ Feet - Wide Open,3,8
22,2021-11-17,1610612745,Houston Rockets,4,6+ Feet - Wide Open,2,8
16,2021-11-07,1610612760,Oklahoma City Thunder,3,6+ Feet - Wide Open,4,8


In [41]:
full_df.to_csv('shot_types_found/shot_types.csv', index=False)

In [42]:
full_df.FG3A.sum()

420

In [43]:
full_df.groupby('SHOT_TYPE').FG3A.sum()

SHOT_TYPE
2-4 Feet - Tight         4
4-6 Feet - Open        116
6+ Feet - Wide Open    300
Name: FG3A, dtype: int64

In [44]:
full_df.loc[full_df.SHOT_TYPE == '2-4 Feet - Tight']

,GAME_DATE,TEAM_ID,TEAM_NAME,PERIOD,SHOT_TYPE,FG3M,FG3A
11,2021-11-03,1610612752,New York Knicks,3,2-4 Feet - Tight,0,1
14,2021-11-05,1610612751,Brooklyn Nets,4,2-4 Feet - Tight,0,2
42,2021-12-10,1610612751,Brooklyn Nets,1,2-4 Feet - Tight,0,1


In [47]:
full_df.groupby(['TEAM_NAME', 'SHOT_TYPE']).FG3A.sum()

TEAM_NAME              SHOT_TYPE          
Atlanta Hawks          4-6 Feet - Open         4
                       6+ Feet - Wide Open     8
Boston Celtics         4-6 Feet - Open        12
Brooklyn Nets          2-4 Feet - Tight        3
                       4-6 Feet - Open        10
Charlotte Hornets      6+ Feet - Wide Open    10
Chicago Bulls          6+ Feet - Wide Open    29
Cleveland Cavaliers    4-6 Feet - Open         8
                       6+ Feet - Wide Open    25
Dallas Mavericks       6+ Feet - Wide Open    20
Denver Nuggets         4-6 Feet - Open         8
                       6+ Feet - Wide Open     5
Detroit Pistons        6+ Feet - Wide Open    18
Golden State Warriors  6+ Feet - Wide Open     1
Houston Rockets        6+ Feet - Wide Open     8
Indiana Pacers         4-6 Feet - Open        12
                       6+ Feet - Wide Open     9
LA Clippers            6+ Feet - Wide Open     3
Los Angeles Lakers     6+ Feet - Wide Open    18
Memphis Grizzlies      4-6